# Plant Leaf Species Classification from Images

In [179]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder

We bring in all the tools needed.

os to work with folders and files.
cv2 (OpenCV) to read and resize images.
numpy for numerical operations.
hog to extract Histogram of Oriented Gradients features.
SVC is the Support Vector Classifier from scikit-learn.
train_test_split to divide data into training/testing sets.
GridSearchCV to tune model parameters (used later).
classification_report and confusion_matrix to evaluate results.
matplotlib.pyplot and seaborn to plot images and results.


HOG is a feature extraction technique commonly used in computer vision, especially for object detection and image classification.

Instead of using raw pixels (which are sensitive to lighting, size, and small changes), HOG captures how the gradient directions (edges and shapes) are distributed in localized parts of the image.

How does HOG work?

Gradient Computation:
For each pixel in the image, calculate the gradient (how intensity changes) in the x and y directions.
This tells us the direction and strength of edges.

Orientation Binning:
The image is divided into small connected regions called cells (e.g., 16x16 pixels).
For each cell, build a histogram of gradient directions weighted by gradient magnitude (edge strength).

Block Normalization:
To improve illumination and contrast invariance, group cells into larger blocks (e.g., 2x2 cells).
Normalize histograms within blocks to make features more robust.

Feature Vector Creation:
Concatenate normalized histograms from all blocks into a single long vector — this is the HOG feature vector.

In [180]:
IMAGE_SIZE = (128, 128)

In [181]:
dataset_path = r"E:\workspace_machine_Learning\Support Vector Machine\Input"

In [182]:
def load_images(dataset_path):
    features = []
    labels = []
    class_names = os.listdir(dataset_path)
    print(f"Classes found: {class_names}")

    for label in class_names:
        class_folder = os.path.join(dataset_path, label)
        for img_file in os.listdir(class_folder):
            img_path = os.path.join(class_folder,img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, IMAGE_SIZE)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Extract HOG features
            hog_features = hog(
                                 gray,
                                 orientations=9,             # try increasing to 12 or 18
                                 pixels_per_cell=(8, 8),     # try smaller cells
                                 cells_per_block=(2, 2),     # increase to (3, 3)
                                 block_norm='L2-Hys',
                                 feature_vector=True
                        )
            features.append(hog_features)
            labels.append(label)
    return np.array(features), np.array(labels)

In [183]:
features, labels = load_images(dataset_path)

Classes found: ['Alstonia Scholaris', 'Arjun', 'Bael', 'Basil', 'Chinar', 'Gauva', 'Jamun', 'Jatropha', 'Lemon', 'Mango', 'Pomegranate', 'Pongamia Pinnata']


In [184]:
features

array([[0.284911  , 0.22905358, 0.23540757, ..., 0.14784574, 0.        ,
        0.        ],
       [0.28592319, 0.        , 0.1899429 , ..., 0.07452851, 0.11783993,
        0.05555028],
       [0.22018113, 0.05421567, 0.20573397, ..., 0.27052719, 0.11166373,
        0.        ],
       ...,
       [0.29597961, 0.14132783, 0.06370376, ..., 0.0475218 , 0.02504619,
        0.        ],
       [0.32126727, 0.06491887, 0.12317489, ..., 0.        , 0.07145059,
        0.03368213],
       [0.27125821, 0.048754  , 0.03083474, ..., 0.02951037, 0.04665999,
        0.07601198]])

In [185]:
labels

array(['Alstonia Scholaris', 'Alstonia Scholaris', 'Alstonia Scholaris',
       'Alstonia Scholaris', 'Alstonia Scholaris', 'Alstonia Scholaris',
       'Alstonia Scholaris', 'Alstonia Scholaris', 'Alstonia Scholaris',
       'Alstonia Scholaris', 'Arjun', 'Arjun', 'Arjun', 'Arjun', 'Arjun',
       'Arjun', 'Arjun', 'Arjun', 'Arjun', 'Arjun', 'Bael', 'Bael',
       'Bael', 'Bael', 'Bael', 'Basil', 'Basil', 'Basil', 'Basil',
       'Basil', 'Chinar', 'Chinar', 'Chinar', 'Chinar', 'Chinar',
       'Chinar', 'Chinar', 'Chinar', 'Chinar', 'Chinar', 'Gauva', 'Gauva',
       'Gauva', 'Gauva', 'Gauva', 'Gauva', 'Gauva', 'Gauva', 'Gauva',
       'Gauva', 'Jamun', 'Jamun', 'Jamun', 'Jamun', 'Jamun', 'Jamun',
       'Jamun', 'Jamun', 'Jamun', 'Jamun', 'Jatropha', 'Jatropha',
       'Jatropha', 'Jatropha', 'Jatropha', 'Jatropha', 'Jatropha',
       'Jatropha', 'Jatropha', 'Jatropha', 'Lemon', 'Lemon', 'Lemon',
       'Lemon', 'Lemon', 'Lemon', 'Lemon', 'Lemon', 'Lemon', 'Lemon',
       'Mango', '

In [186]:
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

In [187]:
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, stratify=labels, random_state=42)

In [188]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(88, 8100) (22, 8100) (88,) (22,)


Variable	   Shape	         Meaning
1. X_train	 (88, 1764)	  88 training images, each with 1764 HOG features
2. X_test	 (22, 1764)	  22 test images, each with 1764 HOG features
3. y_train  	(88,)	    Labels for 88 training images
4. y_test	    (22,)	    Labels for 22 test images

So we have 110 images total (88 train + 22 test) and HOG is extracting 1764 features per image.

In [189]:
# svm = SVC(kernel='linear')  # linear hyperplane to divide classes

In [190]:
svm = SVC(kernel='rbf',class_weight='balanced')   # Non-linear, good for complex decision boundaries

In [191]:
# svm = SVC(kernel='poly', degree=2)  # Polynomial, could help if data has curved separability

In [192]:
svm.fit(X_train,y_train)

SVC(class_weight='balanced')

In [193]:
y_pred = svm.predict(X_test)

In [194]:
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.5454545454545454


In [195]:
print("classification_report: ", classification_report(y_test,y_pred))

classification_report:                precision    recall  f1-score   support

           0       0.17      0.50      0.25         2
           1       1.00      0.50      0.67         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         2
           5       1.00      0.50      0.67         2
           6       1.00      0.50      0.67         2
           7       0.67      1.00      0.80         2
           8       0.50      0.50      0.50         2
           9       1.00      1.00      1.00         2
          10       0.50      0.50      0.50         2
          11       0.00      0.00      0.00         2

    accuracy                           0.55        22
   macro avg       0.57      0.50      0.50        22
weighted avg       0.62      0.55      0.55        22



c:\Users\acer\miniconda3\envs\ds_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\acer\miniconda3\envs\ds_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\acer\miniconda3\envs\ds_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
